In [ ]:
# Import required libraries
import os
import sys
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")

# Add project root to path for imports
project_root = os.path.abspath('.')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import project modules from new structure
from configs.config import Config
from src.utils.data_utils import (
    load_labels_from_csv, 
    SignLanguageDataset, 
    create_data_loaders,
    flip_keypoints,
    transform_keypoints
)
from src.utils.model_utils import (
    create_adjacency_matrix,
    create_model
)
from src.models.model import (
    HGC_LSTM
)
from src.utils.train_utils import (
    train_model
)
from src.utils.visualization_utils import (
    visualize_training_process,
    analyze_model_performance
)

print("✅ All modules imported successfully!")
print(f"📦 Project structure reorganized with modular imports")
print(f"🐍 Python: {sys.version}")
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🔧 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🔧 CUDA Device: {torch.cuda.get_device_name(0)}")
print()

# Initialize configuration
config = Config()
print("🔧 Configuration loaded from configs/config.py")
print(f"📊 Data from: {config.data.input_kp_path}")
print(f"💾 Models save to: {config.model.checkpoint_dir}")
print(f"📈 Plots save to: {config.output.plots_dir}")

In [ ]:
# Import data utilities
from src.utils.data_utils import load_labels_from_csv

# Load labels
video_to_label_mapping, label_to_idx, unique_labels, id_to_label_mapping = load_labels_from_csv(None, config)
num_classes = len(unique_labels)

In [ ]:
# Import data augmentation and dataset utilities
from src.utils.data_utils import flip_keypoints, transform_keypoints, SignLanguageDataset

In [ ]:
# Create datasets with stratified split and augmentation
print("[INFO] Creating datasets...")
keypoints_dir = config.data.keypoints_output_dir

# Set random seed for reproducible splits
np.random.seed(42)

# Get parameters from config
use_strategy = config.data.use_strategy


# Training uses augmentation, validation does not (for fair evaluation)
train_augmentations = getattr(config.data, 'augmentations', [])
val_augmentations = [] 

print(f"Configuration:")
print(f"   Split strategy: {'Stratified' if use_strategy else 'Random'}")
print(f"   Train augmentations: {train_augmentations if train_augmentations else 'None'}")
print(f"   Val augmentations: {val_augmentations if val_augmentations else 'None (for fair evaluation)'}")
if 'translation' in train_augmentations:
    print(f"   Translation range: ±{config.data.translation_range}")
if 'scaling' in train_augmentations:
    print(f"   Scale range: ±{config.data.scale_range}")

train_dataset = SignLanguageDataset(
    keypoints_dir, video_to_label_mapping, label_to_idx, config,
    split_type='train', 
    augmentations=train_augmentations,
    use_strategy=use_strategy
)

val_dataset = SignLanguageDataset(
    keypoints_dir, video_to_label_mapping, label_to_idx, config,
    split_type='val', 
    augmentations=val_augmentations,
    use_strategy=use_strategy
)

print(f"\n[INFO] Dataset summary:")
print(f"  Total classes: {len(unique_labels)}")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")
print(f"  Strategy: {'Stratified' if use_strategy else 'Random'} split")

In [ ]:
# Import data loader utility
from src.utils.data_utils import create_data_loaders

# Create data loaders
print("\n[INFO] Creating data loaders...")
train_loader, val_loader = create_data_loaders(train_dataset, val_dataset, config)

In [ ]:
# Import model utilities
from src.utils.model_utils import create_adjacency_matrix

A = create_adjacency_matrix(config)
print(f"[INFO] Adjacency matrix shape: {A.shape}")
print(f"[INFO] Number of vertices: {config.hgc_lstm.num_vertices}")

In [ ]:
# Import model classes and utilities
from src.utils.model_utils import create_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model(config, A, num_classes, device)

In [ ]:
print("[INFO] Starting training...")
print(f"[INFO] Training configuration:")
print(f"  - Epochs: {config.training.num_epochs}")
print(f"  - Batch size: {config.training.batch_size}")
print(f"  - Learning rate: {config.training.learning_rate}")
print(f"  - Optimizer: {config.training.optimizer}")
print(f"  - Scheduler: {config.training.scheduler}")
print(f"  - Early stopping patience: {config.training.early_stopping_patience}")
history = train_model(model, train_loader, val_loader, config, device)

# Training Process Visualization

This section provides essential visualization of the training process including:
- **Loss curves**: Training and validation loss over epochs
- **Accuracy curves**: Training and validation accuracy over epochs  
- **Confusion matrix**: Model performance analysis on validation set

In [ ]:
# Import visualization utilities
from src.utils.visualization_utils import visualize_training_process

# Visualize training process
print("[INFO] Generating training process visualization...")
training_fig = visualize_training_process(history, config, save_plots=True)

In [ ]:
# Import model performance analysis
from src.utils.visualization_utils import analyze_model_performance
from src.utils.model_utils import load_model
model_save_path = os.path.join(config.model.checkpoint_dir, config.model.save_name)
# model_save_path = os.path.join(config.model.checkpoint_dir, "best_hgc_lstm_321415.pth")
model = load_model(model_save_path, config)
# Analyze model performance
print("[INFO] Analyzing model performance...")
performance_results = analyze_model_performance(model, val_loader, device, config, unique_labels, id_to_label_mapping)

# Dual Attention Weights Visualization

This section visualizes the **dual attention mechanism** from the HGC-LSTM model:

- **Joint Attention**: Shows which keypoints (spatial) the model focuses on after GCN layers
- **Temporal Attention**: Shows which time steps the model focuses on after LSTM layers

The model architecture: **GCN → GCN → Joint Attention → LSTM → Temporal Attention → Classification**

In [ ]:
# Import the new dual attention visualization functions
from src.utils.visualization_utils import visualize_dual_attention_weights, visualize_attention_heatmap

# Visualize dual attention weights (joint + temporal)
print("[INFO] Visualizing dual attention weights...")
print("This shows both:")
print("  - Joint Attention: Which keypoints are important (after GCN)")
print("  - Temporal Attention: Which time steps are important (after LSTM)")

dual_attention_fig = visualize_dual_attention_weights(model, val_loader, device, config, num_samples=3)

In [ ]:
# Visualize joint attention as detailed heatmap
print("[INFO] Visualizing joint attention heatmap...")
print("This shows the actual attention weights from the joint attention layer")
print("(Time steps × Keypoints) - directly from model's attention mechanism")

joint_attention_heatmap_fig = visualize_attention_heatmap(model, val_loader, device, config, num_samples=2)

In [ ]:
from scripts.inference import predict_from_video
import glob
from src.utils.detector import MediaPipeProcessor
test_dir = "data/datatest"  # Replace with your video path
videos = glob.glob(os.path.join(test_dir, "*.mp4"))
count = 0
config = Config()
processor = MediaPipeProcessor(config)
for video_path in videos:
    filename = os.path.basename(video_path)
    number = int(filename.split('_')[1].split('.')[0])
    label, res = predict_from_video(model, processor, id_to_label_mapping, config, device, test_dir+'/'+filename, thresh_hold=0.2)
    print(f"{number}: {res}")
    if number == res:
        count += 1
print(f"{count}/{len(videos)} videos predicted correctly.")